In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, avg

# Start Spark session (if not already started)
spark = SparkSession.builder.appName("SentimentAgg").getOrCreate()

# Set storage account config
spark.conf.set(
    "fs.azure.account.key.ds562team9datalake.blob.core.windows.net",
    "KXg2Djg7uRevBSpPNIVnKw/N6HpqBh+kJwDX07wkywbpU2joMZdTIBOXk30EoMMxH2d8wwb+9j0g+AStO60IWw=="
)

df_sentiments = spark.read.csv(
    "wasbs://silver@ds562team9datalake.blob.core.windows.net/CSV_files/Final_sentiments.csv",
    header=True,
    inferSchema=True
)

# Print schema
df_sentiments.printSchema()
from pyspark.sql.functions import weekofyear, year

df_weekly_agg = df_sentiments.groupBy(
    year("date").alias("year"),
    weekofyear("date").alias("week")
).agg(
    avg("Close").alias("avg_close"),
    avg("tweet_sentiment").alias("avg_tweet_sentiment"),
    avg("news_sentiment").alias("avg_news_sentiment")
)

df_weekly_agg.show()

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, mean, isnan, when

# Start Spark session
spark = SparkSession.builder.appName("ModelInputCheck").getOrCreate()

# Set Azure config
spark.conf.set(
    "fs.azure.account.key.ds562team9datalake.blob.core.windows.net",
    "KXg2Djg7uRevBSpPNIVnKw/N6HpqBh+kJwDX07wkywbpU2joMZdTIBOXk30EoMMxH2d8wwb+9j0g+AStO60IWw=="
)

# Load the CSV
df_model = spark.read.csv(
    "wasbs://silver@ds562team9datalake.blob.core.windows.net/CSV_files/Model_input_file.csv",
    header=True,
    inferSchema=True
)

# Print schema
df_model.printSchema()

# Show sample data
df_model.show(5, truncate=False)

# -------------------------------
# 📊 2. SUMMARY STATISTICS
# -------------------------------
df_model.describe().show()

# -------------------------------
# 📈 3. SIMPLE AGGREGATION EXAMPLE
# -------------------------------
from pyspark.sql.functions import avg

df_model_agg = df_model.agg(
    avg("tweet_sentiment").alias("avg_tweet_sentiment"),
    avg("news_sentiment").alias("avg_news_sentiment"),
    avg("sma_5").alias("avg_sma_5"),
    avg("volatility_3d").alias("avg_volatility_3d")
)

df_model_agg.show()

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, mean, isnan, when, avg, year

# Start Spark session
spark = SparkSession.builder.appName("CorrelationDataCheck").getOrCreate()

# Set Azure config
spark.conf.set(
    "fs.azure.account.key.ds562team9datalake.blob.core.windows.net",
    "KXg2Djg7uRevBSpPNIVnKw/N6HpqBh+kJwDX07wkywbpU2joMZdTIBOXk30EoMMxH2d8wwb+9j0g+AStO60IWw=="
)

# Load the CSV
df_corr = spark.read.csv(
    "wasbs://silver@ds562team9datalake.blob.core.windows.net/CSV_files/Spearman_Pearson_correlation.csv",
    header=True,
    inferSchema=True
)

# Print schema
df_corr.printSchema()

# Show first 5 rows
df_corr.show(5, truncate=False)

# -------------------------------
# 📊 SUMMARY STATISTICS
# -------------------------------
df_corr.describe().show()

# -------------------------------
# 📈 AGGREGATE SELECTED METRICS
# -------------------------------
df_corr_agg = df_corr.agg(
    avg("tweet_sentiment").alias("avg_tweet_sentiment"),
    avg("news_sentiment").alias("avg_news_sentiment"),
    avg("volatility_3d").alias("avg_volatility_3d"),
    avg("rsi_14").alias("avg_rsi_14")
)
df_corr_agg.show()

# -------------------------------
# 📅 GROUP BY YEAR
# -------------------------------
df_corr_by_year = df_corr.groupBy(year("date").alias("year")).agg(
    avg("Close").alias("avg_close"),
    avg("tweet_sentiment").alias("avg_tweet_sentiment"),
    avg("news_sentiment").alias("avg_news_sentiment"),
    avg("volatility_3d").alias("avg_volatility_3d"),
    avg("rsi_14").alias("avg_rsi_14")
)
df_corr_by_year.orderBy("year").show()